In [2]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession, Row
import pandas as pd


In [3]:
spark = SparkSession.builder.appName("rdd-app").config("spark.config.option", "value").getOrCreate()
scfg = SparkConf().setAppName('rdd-app')
sc = spark.sparkContext

In [4]:
import string

text_file = '/user/student/shakespeare/tragedy/hamlet.txt'
text = sc.textFile(text_file)

In [5]:
def strip_punc(s):
    return s.translate(str.maketrans('', '', string.punctuation)).split(' ')

def search_word_in_line(word):
    count = 1
    for line in text.collect():
        if word in strip_punc(line):
            print('{}. {}'.format(count, line))
        count += 1

In [6]:
flatmap = text.flatMap(lambda line: line.translate(str.maketrans('', '', string.punctuation)).split(' '))
map = flatmap.map(lambda word: (word, 1))
reduced = map.reduceByKey(lambda a, b: a + b)

In [7]:
counts = text.flatMap(lambda line: line.translate(str.maketrans('', '', string.punctuation)).split(' '))\
             .map(lambda word: (word, 1))\
             .reduceByKey(lambda a, b: a + b)    

In [8]:
word = "purpose"
for count in reduced.collect():
    # kv = str(count).translate(str.maketrans('', '', string.punctuation)).split(' ')
    kv = strip_punc(str(count))
    if word == kv[0]:
        print('Found \'{}\' occurs \'{}\' times'.format(kv[0], kv[1])) 
        search_word_in_line(word)
        break

Found 'purpose' occurs '11' times
2599.     Why, any thing, but to the purpose. You were sent
2926.     Black as his purpose, did the night resemble
3216.     And drive his purpose on to these delights.
3540.     from the purpose of playing, whose end, both at the
3909.     The passion ending, doth the purpose lose.
4766.     Is but to whet thy almost blunted purpose.
6202.     And, for that purpose, I'll anoint my sword.
6227.     Our purpose may hold there.
6378.     purpose, confess thyself--
7328.     king hold his purpose, I will win for him an I can;
7376.     I am constant to my purpose; they follow the king's


## Manipulating airline performance data

In [10]:

from pyspark.sql.types import Row
from datetime import datetime


In [11]:
data_by_year = '/user/student/airline/1999.csv'
airline_performance = spark.read.option("header", "true").csv(data_by_year)

In [12]:
airline_performance.head()

Row(Year='1999', Month='1', DayofMonth='27', DayOfWeek='3', DepTime='1906', CRSDepTime='1908', ArrTime='2024', CRSArrTime='2005', UniqueCarrier='US', FlightNum='1244', TailNum='N942VJ', ActualElapsedTime='78', CRSElapsedTime='57', AirTime='66', ArrDelay='19', DepDelay='-2', Origin='RIC', Dest='PHL', Distance='198', TaxiIn='3', TaxiOut='9', Cancelled='0', CancellationCode='NA', Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')

In [13]:
airline_performance

DataFrame[Year: string, Month: string, DayofMonth: string, DayOfWeek: string, DepTime: string, CRSDepTime: string, ArrTime: string, CRSArrTime: string, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: string, CancellationCode: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [14]:
from pyspark.sql.types import IntegerType
airline_performance = airline_performance.withColumn("ArrDelay", airline_performance["ArrDelay"].cast(IntegerType()))
airline_performance = airline_performance.withColumn("DepDelay", airline_performance["DepDelay"].cast(IntegerType()))

In [15]:
airline_performance.describe(['ArrDelay']).show()
airline_performance.describe(['DepDelay']).show()

+-------+-----------------+
|summary|         ArrDelay|
+-------+-----------------+
|  count|          5360018|
|   mean|8.246601037533829|
| stddev|33.00760545179995|
|    min|             -194|
|    max|             1724|
+-------+-----------------+

+-------+------------------+
|summary|          DepDelay|
+-------+------------------+
|  count|           5373573|
|   mean|  9.33710084519183|
| stddev|32.268675934546486|
|    min|               -85|
|    max|              1740|
+-------+------------------+



In [16]:
import pyspark.sql.functions as F
airline_performance.select(airline_performance.Dest,F.when(airline_performance.ArrDelay > 0, 1).otherwise(0)).show()

+----+------------------------------------------+
|Dest|CASE WHEN (ArrDelay > 0) THEN 1 ELSE 0 END|
+----+------------------------------------------+
| PHL|                                         1|
| PHL|                                         1|
| PHL|                                         0|
| PHL|                                         1|
| PHL|                                         1|
| PHL|                                         1|
| PHL|                                         0|
| PHL|                                         1|
| PHL|                                         1|
| PHL|                                         1|
| PHL|                                         1|
| PHL|                                         0|
| PHL|                                         0|
| PHL|                                         0|
| PHL|                                         1|
| PHL|                                         0|
| PHL|                                         1|


In [17]:
airline_performance = airline_performance.withColumn('ArrDelayCount',F.when(airline_performance.ArrDelay > 0, 1).otherwise(0))
airline_performance = airline_performance.withColumn('DepDelayCount',F.when(airline_performance.DepDelay > 0, 1).otherwise(0))
airline_performance.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+-------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelayCount|DepDelayCount|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+-------------+
|1999|    

In [29]:
#Most Depature Delay Airport 1999
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','Origin')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+------+------+
|Year|Origin| count|
+----+------+------+
|1999|   ATL|144118|
+----+------+------+
only showing top 1 row



In [30]:
#Least Depature Delay Airport 1999
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','Origin')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+------+-----+
|Year|Origin|count|
+----+------+-----+
|1999|   LFT|    1|
+----+------+-----+
only showing top 1 row



In [20]:
#Most Arrival Delay Airport 1999
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','Dest')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+----+------+
|Year|Dest| count|
+----+----+------+
|1999| ORD|143605|
+----+----+------+
only showing top 1 row



In [21]:
#least Arrival Delay Airport 1999
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','Dest')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+----+-----+
|Year|Dest|count|
+----+----+-----+
|1999| LFT|    2|
+----+----+-----+
only showing top 1 row



In [32]:
#Most Arrival Delay Flight 1999
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|1999|           WN|      198| 1163|
+----+-------------+---------+-----+
only showing top 1 row



In [26]:
#Least Arrival Delay Flight 1999
airline_performance\
.filter(F.col('ArrDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|1999|           WN|     1861|    1|
+----+-------------+---------+-----+
only showing top 1 row



In [27]:
#Most Departure Delay Flight 1999
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|1999|           WN|     1702| 1116|
+----+-------------+---------+-----+
only showing top 1 row



In [28]:
#Least Departure Delay Flight 1999
airline_performance\
.filter(F.col('DepDelayCount') == 1)\
.groupBy('Year','UniqueCarrier','FlightNum')\
.count()\
.orderBy('count',ascending=True)\
.show(1)

+----+-------------+---------+-----+
|Year|UniqueCarrier|FlightNum|count|
+----+-------------+---------+-----+
|1999|           WN|     1866|    1|
+----+-------------+---------+-----+
only showing top 1 row



In [38]:
airline_performance\
.groupBy('Year','UniqueCarrier')\
.agg(F.mean('ArrDelay'))\
.show()

+----+-------------+------------------+
|Year|UniqueCarrier|     avg(ArrDelay)|
+----+-------------+------------------+
|1999|           WN|7.4698429124716865|
|1999|           NW| 4.389880451576387|
|1999|           UA| 9.882414898898416|
|1999|           TW| 6.008880764633835|
|1999|           US|10.441609242053934|
|1999|           DL| 6.678932548316908|
|1999|           HP|13.208234475185789|
|1999|           CO| 8.480354162613779|
|1999|           AS|11.003722522258553|
|1999|           AA| 8.918916508042582|
+----+-------------+------------------+



In [39]:
airline_performance\
.groupBy('Year','UniqueCarrier')\
.agg(F.mean('DepDelay'))\
.show()

+----+-------------+------------------+
|Year|UniqueCarrier|     avg(DepDelay)|
+----+-------------+------------------+
|1999|           WN|10.093180644314879|
|1999|           NW| 6.718094058226521|
|1999|           UA| 9.899378991741754|
|1999|           TW|  8.20298647446099|
|1999|           US|10.408952370620462|
|1999|           DL| 8.062052674456261|
|1999|           HP|12.003973885892705|
|1999|           CO| 9.118107321556213|
|1999|           AS|10.887364496397627|
|1999|           AA|  9.84248138823221|
+----+-------------+------------------+

